# Домашнее задание 7

Сегодня будем решать задачу _машинного перевода_ с помощью RNN.

1. Построим RNN, обучим на текстах.
2. Построим bi-directional RNN, обучим, сравним качество.

Стоит отметить, что RNN - это не самый популярный и надежный метод из-за проблем с затуханием и взрывом градиентов, а также ограниченной способности захватывать долгосрочные зависимости в тексте.
Более улучшенные и эффективные модели перевода (такие как LSTM, GRU и трансформеры) вы узнаете в блоке NLP.


In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import numpy as np

In [2]:
# Загружаем датасет

In [26]:
def load_pairs(file_path):
    pairs = []
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            pair = line.strip().split("\t")
            pairs.append(pair)
    return pairs


pairs = load_pairs("data/pairs.txt")

In [27]:
pairs[200:205]

[['Back off.', 'Посторонитесь.'],
 ['Be a man.', 'Будь мужчиной!'],
 ['Be brave.', 'Будь храбр.'],
 ['Be brief.', 'Будь краток.'],
 ['Be quiet.', 'Тихо.']]

In [5]:
# Делаем нужные предобработки для задачи перевода

In [28]:
# Определение специальных токенов
PAD_TOKEN = "<PAD>"
EOS_TOKEN = "<EOS>"
SOS_TOKEN = "<SOS>"
UNK_TOKEN = "<UNK>"


# Функция токенизации предложения: приводит все символы к нижнему регистру и разбивает предложение на слова
def tokenize(sentence):
    return sentence.lower().split()


# Функция для построения словарей для английских и русских слов на основе пар предложений
def build_vocab(pairs):
    eng_vocab = set()
    rus_vocab = set()
    for eng_sentence, rus_sentence in pairs:
        eng_vocab.update(tokenize(eng_sentence))
        rus_vocab.update(tokenize(rus_sentence))
    return eng_vocab, rus_vocab


# Функция для создания отображений слово -> индекс и индекс -> слово
def create_mappings(vocab):
    vocab = [PAD_TOKEN, SOS_TOKEN, EOS_TOKEN, UNK_TOKEN] + sorted(vocab)
    word2int = {word: i for i, word in enumerate(vocab)}
    int2word = {i: word for word, i in word2int.items()}
    return word2int, int2word


# Создание словарей для английских и русских предложений на основе пар
english_vocab, russian_vocab = build_vocab(pairs)

# Создание отображений с добавлением специальных токенов
eng_word2int, eng_int2word = create_mappings(english_vocab)
rus_word2int, rus_int2word = create_mappings(russian_vocab)

# Печать размеров словарей (с учетом 4 специальных токенов)
print("English vocabulary size:", len(english_vocab) + 4)
print("Russian vocabulary size:", len(russian_vocab) + 4)

# Пример использования: кодирование английского и русского предложения
eng_example = "Who are you"
rus_example = "как ты"

# Кодирование с учетом UNK_TOKEN для неизвестных слов
eng_encoded = np.array(
    [eng_word2int.get(word, eng_word2int[UNK_TOKEN]) for word in tokenize(eng_example)],
    dtype=np.int32,
)
rus_encoded = np.array(
    [rus_word2int.get(word, rus_word2int[UNK_TOKEN]) for word in tokenize(rus_example)],
    dtype=np.int32,
)

print("English text encoded:", eng_encoded)
print("Russian text encoded:", rus_encoded)

# Декодирование: восстановление текста из кодов
decoded_eng = " ".join([eng_int2word[i] for i in eng_encoded])
decoded_rus = " ".join([rus_int2word[i] for i in rus_encoded])

print("Decoded English:", decoded_eng)
print("Decoded Russian:", decoded_rus)


# Определение класса датасета для перевода
class TranslationDataset(Dataset):
    def __init__(self, pairs, eng_word2int, rus_word2int):
        self.pairs = pairs
        self.eng_word2int = eng_word2int
        self.rus_word2int = rus_word2int

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        eng, rus = self.pairs[idx]
        # Кодирование английского предложения и добавление EOS токена
        eng_tensor = torch.tensor(
            [
                self.eng_word2int.get(word, self.eng_word2int[UNK_TOKEN])
                for word in tokenize(eng)
            ]
            + [self.eng_word2int[EOS_TOKEN]],
            dtype=torch.long,
        )
        # Кодирование русского предложения и добавление EOS токена
        rus_tensor = torch.tensor(
            [
                self.rus_word2int.get(word, self.rus_word2int[UNK_TOKEN])
                for word in tokenize(rus)
            ]
            + [self.rus_word2int[EOS_TOKEN]],
            dtype=torch.long,
        )
        return eng_tensor, rus_tensor


# Функция для объединения батчей: паддинг (дополнение) предложений до одной длины в батче
def collate_fn(batch):
    eng_batch, rus_batch = zip(*batch)
    eng_batch_padded = pad_sequence(
        eng_batch, batch_first=True, padding_value=eng_word2int[PAD_TOKEN]
    )
    rus_batch_padded = pad_sequence(
        rus_batch, batch_first=True, padding_value=rus_word2int[PAD_TOKEN]
    )
    return eng_batch_padded, rus_batch_padded


# Создание экземпляра датасета и загрузчика данных
translation_dataset = TranslationDataset(pairs, eng_word2int, rus_word2int)
batch_size = 64
translation_dataloader = DataLoader(
    translation_dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    collate_fn=collate_fn,
)

# Печать информации о количестве образцов и батчей в датасете
print("Translation samples: ", len(translation_dataset))
print("Translation batches: ", len(translation_dataloader))

English vocabulary size: 34195
Russian vocabulary size: 86949
English text encoded: [33425  2292 34085]
Russian text encoded: [25873 77975]
Decoded English: who are you
Decoded Russian: как ты
Translation samples:  323711
Translation batches:  5057


## Простая RNN

Начнем свои эксперименты с простой `RNN` - без bidirectional и с одним слоем.

### Задание №1

Добавьте недостающие части в класс `Encoder` и сдайте в ЛМС код класса.

In [11]:
class Encoder(nn.Module):
    def __init__(
        self, vocab_size: int, embed_size: int, hidden_size: int, num_layers: int = 1
    ):
        super().__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.RNN(embed_size, hidden_size, num_layers=num_layers, batch_first=True)

    def forward(self, x):
        x = torch.flip(x, [1])
        embedded = self.embedding(x)
        outputs, hidden = self.rnn(embedded)
        return outputs, hidden

### Задание №2

Добавьте недостающий код в `Decoder` и сдайте в ЛМС код класса.

In [15]:
class Decoder(nn.Module):
    def __init__(
        self, vocab_size: int, embed_size: int, hidden_size: int, num_layers: int = 1
    ):
        super().__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.RNN(embed_size, hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(in_features=hidden_size, out_features=vocab_size)

    def forward(self, x: torch.Tensor, hidden: torch.Tensor | None):
        out = self.embedding(x)
        out, hidden = self.rnn(out, hidden)
        out = self.fc(out).reshape(out.size(0), -1)
        return out, hidden

In [8]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cuda', index=0)

In [16]:
eng_vocab_size = len(eng_word2int)
rus_vocab_size = len(rus_word2int)
embed_size = 256
hidden_size = 512
num_layers = 1

torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
encoder = Encoder(eng_vocab_size, embed_size, hidden_size, num_layers).to(DEVICE)
decoder = Decoder(rus_vocab_size, embed_size, hidden_size, num_layers).to(DEVICE)

In [17]:
def translate(encoder, decoder, sentence, eng_word2int, rus_int2word, max_length=15):
    # Переводим модели в режим оценки (inference)
    encoder.eval()
    decoder.eval()

    # Отключаем вычисление градиентов для ускорения и уменьшения использования памяти
    with torch.inference_mode():
        # Преобразуем входное предложение в тензор и добавляем EOS токен в конце
        input_tensor = torch.tensor(
            [eng_word2int[word] for word in tokenize(sentence)]
            + [eng_word2int[EOS_TOKEN]],
            dtype=torch.long,
        )
        input_tensor = input_tensor.view(1, -1).to(DEVICE)  # batch_first=True

        # Пропускаем входное предложение через энкодер
        _, encoder_hidden = encoder(input_tensor)
        # Инициализируем скрытое состояние декодера скрытым состоянием энкодера
        decoder_hidden = encoder_hidden

        decoded_words = []
        last_word = torch.tensor([[eng_word2int[SOS_TOKEN]]]).to(DEVICE)
        for _ in range(max_length):
            # Пропускаем последний предсказанный токен через декодер
            logits, decoder_hidden = decoder(last_word, decoder_hidden)
            # Жадный перебор: выбираем токен с максимальной вероятностью - можно было и с температурой, попробуйте в качестве эксперименте
            next_token = logits.argmax(dim=1)
            last_word = next_token.unsqueeze(0).to(DEVICE)
            if next_token.item() == rus_word2int[EOS_TOKEN]:
                break
            else:
                decoded_words.append(rus_int2word.get(next_token.item()))

    # Возвращаем переведенные слова как строку
    return " ".join(decoded_words)

In [18]:
sentence = "just do it"
translated_sentence = translate(encoder, decoder, sentence, eng_word2int, rus_int2word)
print("Translated:", translated_sentence)

Translated: еноты. сформулировать прабабушек? бесплатно засохнет. опухшие целью подавала сочувствии приезжаешь. было? пища? столб. мвф уголках


Пока перевод получается странный, но мы ведь пока ничего не обучали.

In [19]:
import random

import torch.nn as nn

# Функция потерь (исключая паддинг)
loss_fn = nn.CrossEntropyLoss(ignore_index=eng_word2int[PAD_TOKEN])

# Оптимизаторы
encoder_optimizer = optim.AdamW(encoder.parameters())
decoder_optimizer = optim.AdamW(decoder.parameters())

# Количество эпох
num_epochs = 1

# Тренировочный цикл
encoder.train()
decoder.train()

for epoch in range(num_epochs):
    for i, (input_tensor, target_tensor) in enumerate(translation_dataloader):
        input_tensor, target_tensor = input_tensor.to(DEVICE), target_tensor.to(DEVICE)

        # Обнуление градиентов обоих оптимизаторов
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        target_length = target_tensor.size(1)

        # Энкодер
        _, encoder_hidden = encoder(input_tensor)

        # Декодер
        decoder_input = torch.full(
            (batch_size, 1), eng_word2int[SOS_TOKEN], dtype=torch.long
        ).to(DEVICE)
        decoder_hidden = encoder_hidden

        # Случайный выбор индекса слова из целевой последовательности
        random_word_index = random.randint(0, target_length - 1)

        loss = torch.tensor(0.0, device=DEVICE, requires_grad=True)
        for di in range(target_length):
            logits, _ = decoder(decoder_input, decoder_hidden)

            # Вычисление потерь только для случайно выбранного слова
            loss = loss + loss_fn(logits, target_tensor[:, di])

            decoder_input = target_tensor[:, di].reshape(
                batch_size, 1
            )  # Teacher forcing (принудительное обучение)

        # Обратное распространение ошибки и шаг оптимизации
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()

        if i % 100 == 0:
            # Печать потерь каждые 100 батчей
            print(f"Epoch {epoch}, Batch {i}, Loss: {loss.item() / target_length:.4f}")

Epoch 0, Batch 0, Loss: 11.3994
Epoch 0, Batch 100, Loss: 6.5943
Epoch 0, Batch 200, Loss: 5.2954
Epoch 0, Batch 300, Loss: 6.2373
Epoch 0, Batch 400, Loss: 5.5826
Epoch 0, Batch 500, Loss: 5.2326
Epoch 0, Batch 600, Loss: 5.8907
Epoch 0, Batch 700, Loss: 5.2402
Epoch 0, Batch 800, Loss: 5.8356
Epoch 0, Batch 900, Loss: 5.4579
Epoch 0, Batch 1000, Loss: 4.7024
Epoch 0, Batch 1100, Loss: 4.4864
Epoch 0, Batch 1200, Loss: 4.8597
Epoch 0, Batch 1300, Loss: 4.6255
Epoch 0, Batch 1400, Loss: 5.3215
Epoch 0, Batch 1500, Loss: 4.9888
Epoch 0, Batch 1600, Loss: 5.0402
Epoch 0, Batch 1700, Loss: 5.5525
Epoch 0, Batch 1800, Loss: 6.6702
Epoch 0, Batch 1900, Loss: 5.4752
Epoch 0, Batch 2000, Loss: 3.9696
Epoch 0, Batch 2100, Loss: 4.0935
Epoch 0, Batch 2200, Loss: 4.3770
Epoch 0, Batch 2300, Loss: 4.1122
Epoch 0, Batch 2400, Loss: 4.4943
Epoch 0, Batch 2500, Loss: 4.4967
Epoch 0, Batch 2600, Loss: 5.0038
Epoch 0, Batch 2700, Loss: 4.4875
Epoch 0, Batch 2800, Loss: 4.7462
Epoch 0, Batch 2900, Loss

### Задание №3
Попробуйте перевести предложение "Where is Tom?".
Сдайте в ЛМС перевод.

In [20]:
sentence = "Where is Tom?"
translated_sentence = translate(encoder, decoder, sentence, eng_word2int, rus_int2word)
print("Translated:", translated_sentence)

Translated: где припарковал томом?


## Bidirectional RNN

Теперь попробуем использовать двунаправленную RNN (bidirectional RNN) в энкодере,
что позволяет модели учитывать информацию из обеих сторон последовательности — как слева направо, так и справа налево.

Декодер остается односторонним.

### Задание №4

Допишите недостающий код в `Encoder` и сдайте на в ЛМС код класса.

In [29]:
class Encoder(nn.Module):
    def __init__(
        self, vocab_size: int, embed_size: int, hidden_size: int, num_layers: int = 1
    ):
        super().__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.RNN(embed_size, hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True)
        # Добавьте двунаправленную RNN

    def forward(self, x: torch.Tensor):
        embedded = self.embedding(x)
        outputs, hidden = self.rnn(embedded)
        # Двунаправленная RNN возвращает два скрытых состояния: одно для каждого направления.
        # Объединяем их в одно скрытое состояние.
        hidden = torch.cat((hidden[0, :, :], hidden[1, :, :]), dim=1).unsqueeze(0)
        return outputs, hidden

### Задание №5

Допишите класс `Decoder` и сдайте в ЛМС его реализацию.

In [30]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers=1):
        super().__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.RNN(embed_size, hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden):
        out = self.embedding(x)
        out, hidden = self.rnn(out, hidden)
        out = self.fc(out).reshape(out.size(0), -1)
        return out, hidden

In [31]:
eng_vocab_size = len(eng_word2int)
ita_vocab_size = len(rus_word2int)
embed_size = 256
hidden_size = 512
num_layers = 1

encoder = Encoder(eng_vocab_size, embed_size, hidden_size, num_layers).to(DEVICE)
decoder = Decoder(ita_vocab_size, embed_size, hidden_size * 2, num_layers).to(DEVICE)

In [32]:
import random

import torch.nn as nn
import torch.optim as optim

loss_fn = nn.CrossEntropyLoss(ignore_index=eng_word2int[PAD_TOKEN])

encoder_optimizer = optim.AdamW(encoder.parameters())
decoder_optimizer = optim.AdamW(decoder.parameters())

num_epochs = 1

encoder.train()
decoder.train()

for epoch in range(num_epochs):
    for i, (input_tensor, target_tensor) in enumerate(translation_dataloader):
        input_tensor, target_tensor = input_tensor.to(DEVICE), target_tensor.to(DEVICE)

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        target_length = target_tensor.size(1)

        _, encoder_hidden = encoder(input_tensor)

        decoder_input = torch.full(
            (batch_size, 1), eng_word2int[SOS_TOKEN], dtype=torch.long
        ).to(DEVICE)
        decoder_hidden = encoder_hidden

        random_word_index = random.randint(0, target_length - 1)

        loss = torch.tensor(0.0, device=DEVICE, requires_grad=True)
        for di in range(target_length):
            logits, decoder_hidden = decoder(decoder_input, decoder_hidden)

            loss = loss + loss_fn(logits, target_tensor[:, di])

            decoder_input = target_tensor[:, di].reshape(batch_size, 1)

        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()

        if i % 100 == 0:
            print(f"Epoch {epoch}, Batch {i}, Loss: {loss.item() / target_length:.4f}")

Epoch 0, Batch 0, Loss: 11.3901
Epoch 0, Batch 100, Loss: 6.0357
Epoch 0, Batch 200, Loss: 6.4146
Epoch 0, Batch 300, Loss: 5.0133
Epoch 0, Batch 400, Loss: 6.5024
Epoch 0, Batch 500, Loss: 5.2549
Epoch 0, Batch 600, Loss: 5.1522
Epoch 0, Batch 700, Loss: 6.5301
Epoch 0, Batch 800, Loss: 7.5251
Epoch 0, Batch 900, Loss: 6.1641
Epoch 0, Batch 1000, Loss: 4.4248
Epoch 0, Batch 1100, Loss: 5.8862
Epoch 0, Batch 1200, Loss: 5.9552
Epoch 0, Batch 1300, Loss: 4.7868
Epoch 0, Batch 1400, Loss: 5.8457
Epoch 0, Batch 1500, Loss: 4.4775
Epoch 0, Batch 1600, Loss: 4.7723
Epoch 0, Batch 1700, Loss: 5.9154
Epoch 0, Batch 1800, Loss: 6.0398
Epoch 0, Batch 1900, Loss: 5.6283
Epoch 0, Batch 2000, Loss: 6.1216
Epoch 0, Batch 2100, Loss: 4.5640
Epoch 0, Batch 2200, Loss: 4.4292
Epoch 0, Batch 2300, Loss: 4.8590
Epoch 0, Batch 2400, Loss: 5.0518
Epoch 0, Batch 2500, Loss: 4.2904
Epoch 0, Batch 2600, Loss: 4.6329
Epoch 0, Batch 2700, Loss: 5.6533
Epoch 0, Batch 2800, Loss: 5.0178
Epoch 0, Batch 2900, Loss

In [33]:
sentence = "just do it"
translated_sentence = translate(encoder, decoder, sentence, eng_word2int, rus_int2word)
print("Translated:", translated_sentence)

Translated: я очень жадный.


Выбить хорошее качество обучения, используя только лишь RNN и небольшой датасет, сложно.

Не забывайте, что в семинаре у нас было ~400 Мб текстов одного языка, а здесь всего лишь 28 Мб и на двух языках.
Можем сделать вывод, что для обучения хорошей модели нужно много текстовых данных.

Помимо этого, для серьезного обучения стоит использовать более продвинутые сети: те же GRU и LSTM покажут себя лучше.
А еще лучше будет работать трансформер, о котором вы узнаете в следующем уроке.